네트워크 분석을 머신러닝과 딥러닝 기법으로 대체하여 법률 분야에서 경찰 범죄 관련 데이터 분석을 수행하는 방법을 설명하겠습니다. 여기서는 주로 네트워크의 노드(예: 변호사, 판사, 사건) 간의 관계를 분석하고, 머신러닝 및 딥러닝을 통해 이 관계를 학습합니다.

1. 데이터 전처리
네트워크 분석을 위한 데이터는 변호사, 판사, 사건 정보와 그들 간의 관계를 포함해야 합니다. 데이터를 불러와 필요한 전처리를 수행합니다.

1.1 데이터 로드 및 전처리

In [ ]:
import pandas as pd

# 데이터 로드
lawyers = pd.read_csv('lawyers.csv')  # 변호사 데이터
judges = pd.read_csv('judges.csv')    # 판사 데이터
cases = pd.read_csv('cases.csv')       # 사건 데이터
relationships = pd.read_csv('relationships.csv')  # 관계 데이터 (변호사-사건, 판사-사건 등)

# 데이터 전처리: 필요에 따라 중복 제거, 결측치 처리 등 수행


2. 피처 생성
변호사와 판사 간의 관계를 기반으로 피처를 생성합니다. 이를 통해 머신러닝 및 딥러닝 모델의 입력 데이터를 준비합니다.

2.1 피처 생성 예시

In [ ]:
import networkx as nx

# 관계 그래프 생성
G = nx.Graph()

# 변호사와 사건 간의 관계 추가
for _, row in relationships.iterrows():
    G.add_edge(row['lawyer_id'], row['case_id'], relationship='represents')

# 판사와 사건 간의 관계 추가
for _, row in relationships.iterrows():
    G.add_edge(row['judge_id'], row['case_id'], relationship='decides')

# 각 노드의 특성을 가져오기
lawyer_features = {node: G.degree(node) for node in lawyers['lawyer_id']}
judge_features = {node: G.degree(node) for node in judges['judge_id']}

# 피처 데이터프레임 생성
features_df = pd.DataFrame({
    'lawyer_id': lawyer_features.keys(),
    'degree': lawyer_features.values()
})

# 판사와 사건의 피처도 비슷하게 생성


3. 머신러닝 모델 학습
생성된 피처를 사용하여 변호사 또는 판사의 행동이나 사건의 결과를 예측하는 머신러닝 모델을 학습합니다.

3.1 머신러닝 예시

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# 레이블 및 특성 설정
X = features_df[['degree']]  # 특성
y = cases['outcome']  # 사건 결과 (예: 원고 승소, 피고 승소)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest 모델 학습
model = RandomForestClassifier()
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


4. 딥러닝 모델 학습
딥러닝을 통해 복잡한 패턴을 학습할 수 있습니다. 여기서는 Keras를 사용하여 신경망을 구축하는 예를 보여줍니다.

4.1 딥러닝 예시

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# 신경망 모델 생성
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # 이진 분류인 경우

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# 예측 및 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy:.4f}')


5. 문장 유사도 분석
문장 유사도 분석을 추가하여 법률 문서 간의 유사도를 측정할 수 있습니다. 이를 위해 BERT를 사용할 수 있습니다.

5.1 문장 유사도 예시

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# BERT 모델 및 토크나이저 초기화
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 문장 임베딩 함수 정의
def get_sentence_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()  # [CLS] 토큰의 임베딩

# 두 문장 간의 유사도 계산
def calculate_similarity(sentence1, sentence2):
    embedding1 = get_sentence_embedding(sentence1)
    embedding2 = get_sentence_embedding(sentence2)
    return cosine_similarity(embedding1, embedding2)[0][0]

# 문장 유사도 측정
similarity = calculate_similarity("사건 설명 1", "사건 설명 2")
print("문장 유사도:", similarity)


6. 요약
이 코드는 법률 분야에서 네트워크 분석을 머신러닝과 딥러닝 기법으로 대체하는 방법을 설명합니다.

데이터 전처리: 변호사, 판사, 사건 데이터 로드 및 전처리.
피처 생성: 네트워크 그래프를 이용하여 노드의 특성(예: 차수)을 생성.
머신러닝 모델 학습: Random Forest를 사용하여 사건 결과 예측.
딥러닝 모델 학습: Keras를 사용하여 신경망 모델 구축 및 학습.
문장 유사도 분석: BERT를 활용하여 문장 간 유사도 측정.